In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet

In [3]:
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
import folium 
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap

#### 2. Import the data located at this link. It has information on people infected with dengue at the district level for 2015 to 2021.

In [4]:
base = open(r'../../_data/data_dengue_peru.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']
charenc

'UTF-8-SIG'

In [5]:
dengue = pd.read_csv( r'../../_data/data_dengue_peru.csv', encoding = charenc, thousands = ",", dtype= {"Ubigeo":"str"})
dengue.head( 5 )

,Año,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos
0,2020,43,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,2.0
1,2021,41,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN
2,2021,42,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN
3,2020,1,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,0.0
4,2020,53,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,2.0


#### 3. Generate ubigeo for Departments and Provinces taking the first two and four numbers.

In [6]:
dengue['Dpt_Ubigeo'] = dengue['Ubigeo'].astype(str).str[0:2]
dengue['Prv_Ubigeo'] = dengue['Ubigeo'].astype(str).str[0:4]

In [7]:
dengue.head(3)

,Año,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos,Dpt_Ubigeo,Prv_Ubigeo
0,2020,43,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,2.0,08,0809
1,2021,41,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN,18,1801
2,2021,42,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN,18,1801


#### 4. Use geopandas to plot the number of cases in 2021 by the district using a continuous legend. Do not forget to indicate the color of NA values. Use this shapefile.

In [8]:
distritos = gpd.read_file(r'../../_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp')
distritos.head(3)

,CCDD,NOMBDEP,CCPP,NOMBPROV,CCDI,NOMBDIST,CAPITAL,UBIGEO,IDPROV,CODIGO,CNT_CCPP,DESCRIPCIO,DESCARGAR,WHATSAPP,CONTACTO,geometry
0,06,CAJAMARCA,05,CONTUMAZA,04,GUZMANGO,GUZMANGO,060504,0605,060504,50,GUZMANGO,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-78.91578 -7.32085, -78.91370 -7.322..."
1,06,CAJAMARCA,11,SAN MIGUEL,05,EL PRADO,EL PRADO,061105,0611,061105,15,EL PRADO,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-79.01679 -6.96945, -79.01581 -6.969..."
2,06,CAJAMARCA,11,SAN MIGUEL,09,NIEPOS,NIEPOS,061109,0611,061109,41,NIEPOS,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-79.21691 -6.85171, -79.21576 -6.851..."


In [9]:
dengue_2021 = dengue[dengue.Año == 2021]
dengue_2021.head(3)

,Año,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos,Dpt_Ubigeo,Prv_Ubigeo
1,2021,41,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN,18,1801
2,2021,42,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN,18,1801
6,2021,41,Dengue,LIMA,INDEPENDENCIA,LIMA,150112,NaN,15,1501


#### 5. Use geopandas to plot the number of cases in 2021by the province using a continuous legend. Do not forget to indicate the color of NA values. Use this shapefile. For this task, you will have to aggregate shapefiles at the province level.

#### 6. Use geopandas to plot the number of cases by the department for all the years using subplots. Every subplot for each year. Do not forget to indicate the color of NA values. Use this shapefile. For this task, you will have to aggregate shapefiles at the department level.

In [24]:
dengue_departamento= dengue_2021.groupby(['Departamento'])['Casos'].sum().reset_index().copy()
dengue_departamento

,Departamento,Casos
0,AM,1830.0
1,AN,26.0
2,AY,1346.0
3,CA,2557.0
4,CU,1341.0
5,HU,2735.0
6,IC,4577.0
7,JU,3168.0
8,LA,1197.0
9,LI,1119.0


In [28]:

distritos['IDDIST'] = distritos['IDDIST'].astype(str).astype(int)

maps = distritos[['IDDIST','DISTRITO', 'geometry']]
data_geo7 = maps.merge(dengue_departamento, on='IDDIST', how='left')
data_geo7

KeyError: 'IDDIST'